In [1]:
import findspark
findspark.init()

In [2]:
import matplotlib.pyplot as plt
import pyspark
import numpy as np
import pandas as pd
from numpy import polyfit
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
#obj2 = spark.read.csv("C:\data\FINAL-2\objetivo2-final.csv",header=True, inferSchema=True, nullValue="NA")
obj2=spark.read.format("csv") \
    .option("quote", "") \
    .option("escape", "\"") \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("encoding","windows-1252") \
    .load("C:\data\FINAL-3\objetivo2-final.csv")

In [4]:
obj2.show(5)

+---+---------+---------------+---------------+----+------+---------+------------+------+----+---------+-------------------+----------------------+
| ID|  Nombres|ApellidoPaterno|ApellidoMaterno|Edad|Genero|      IMC|Departamento|  Zona|Anio|      Mes|CategoriaEnfermedad|SubcategoriaEnfermedad|
+---+---------+---------------+---------------+----+------+---------+------------+------+----+---------+-------------------+----------------------+
|  1|  Celinka|     Dawidowitz|        MacKill|  28|     F| OBESIDAD|       JUNÍN| RURAL|2020|NOVIEMBRE|ENFERMEDAD OBESIDAD|  ADIPOSIDAD LOCALI...|
|  2|    Reina|         Towhey|      Tallquist|  24|     F|   NORMAL|         ICA|URBANA|2021|    MARZO|ENFERMEDAD OBESIDAD|              OBESIDAD|
|  3|    Gerda|      Firebrace|         Harbin|  97|     F| OBESIDAD|      LORETO| RURAL|2021|DICIEMBRE|ENFERMEDAD OBESIDAD|          DISLIPIDEMIA|
|  4|Mireielle|        Yeatman|     Braybrooks|  66|     F| OBESIDAD|  SAN MARTÍN| RURAL|2017|  FEBRERO|ENFERMED

# Selección de datos

In [5]:
#Objetivo 2 - Eliminando columnas
obj2 = obj2.drop('id','Apellidos','Nombres','Categoria','IMC')
obj2.show(10)

#Objetivo 2 - Agrupacion de datos
obj2.createOrReplaceTempView('datos')
query = 'SELECT SubcategoriaEnfermedad, Zona, Departamento, Mes, Anio ,count(*) as Cantidad from datos \
            group by Departamento, Zona, SubcategoriaEnfermedad, Mes, Anio'
obj2_final=spark.sql(query)
obj2_final.show(10)


+---------------+---------------+----+------+------------+------+----+----------+-------------------+----------------------+
|ApellidoPaterno|ApellidoMaterno|Edad|Genero|Departamento|  Zona|Anio|       Mes|CategoriaEnfermedad|SubcategoriaEnfermedad|
+---------------+---------------+----+------+------------+------+----+----------+-------------------+----------------------+
|     Dawidowitz|        MacKill|  28|     F|       JUNÍN| RURAL|2020| NOVIEMBRE|ENFERMEDAD OBESIDAD|  ADIPOSIDAD LOCALI...|
|         Towhey|      Tallquist|  24|     F|         ICA|URBANA|2021|     MARZO|ENFERMEDAD OBESIDAD|              OBESIDAD|
|      Firebrace|         Harbin|  97|     F|      LORETO| RURAL|2021| DICIEMBRE|ENFERMEDAD OBESIDAD|          DISLIPIDEMIA|
|        Yeatman|     Braybrooks|  66|     F|  SAN MARTÍN| RURAL|2017|   FEBRERO|ENFERMEDAD OBESIDAD|  ADIPOSIDAD LOCALI...|
|         Payley|        Larkkem|  84|     M|        PUNO| RURAL|2021|     ENERO|ENFERMEDAD OBESIDAD|       DIABETES TIPO 2|


# Limpieza de datos

In [6]:
#Eliminando valores nulos del dataset
obj2_final = obj2_final.dropna()
obj2_final = obj2_final.filter('Anio != 2022')
obj2_final = obj2_final.where((obj2.Zona == 'RURAL') | (obj2.Zona == 'URBANA'))
obj2_final.show(10)


+----------------------+------+------------+----------+----+--------+
|SubcategoriaEnfermedad|  Zona|Departamento|       Mes|Anio|Cantidad|
+----------------------+------+------------+----------+----+--------+
|  ADIPOSIDAD LOCALI...| RURAL|       CUSCO|   OCTUBRE|2020|       2|
|  SECUELAS DE HIPER...|URBANA|      CALLAO|      MAYO|2017|       6|
|       DIABETES TIPO 2|URBANA|   CAJAMARCA|     ENERO|2019|       8|
|          HIGADO GRASO| RURAL|      LORETO|     JUNIO|2016|       5|
|  SINDROME DE APNEA...| RURAL|     HUÁNUCO|SEPTIEMBRE|2019|       4|
|  SECUELAS DE HIPER...|URBANA| LA LIBERTAD|   FEBRERO|2020|       5|
|          INFERTILIDAD| RURAL|       PASCO|     MARZO|2016|       9|
|  SECUELAS DE HIPER...| RURAL| LA LIBERTAD|   FEBRERO|2021|       3|
|  OTROS TIPOS DE HI...| RURAL|    MOQUEGUA|   OCTUBRE|2017|       6|
|       DIABETES TIPO 2|URBANA|      ANCASH|   OCTUBRE|2020|       7|
+----------------------+------+------------+----------+----+--------+
only showing top 10 

# Integración de datos

In [8]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [9]:
obj2_final = StringIndexer(inputCol='SubcategoriaEnfermedad', outputCol= 'subcat_idx').fit(obj2_final).transform(obj2_final)
obj2_final = StringIndexer(inputCol='Zona', outputCol= 'zona_idx').fit(obj2_final).transform(obj2_final)
obj2_final = StringIndexer(inputCol='Departamento', outputCol= 'dpto_idx').fit(obj2_final).transform(obj2_final)
obj2_final = StringIndexer(inputCol='Mes', outputCol= 'mes_idx').fit(obj2_final).transform(obj2_final)
obj2_final = StringIndexer(inputCol='Anio', outputCol= 'anio_idx').fit(obj2_final).transform(obj2_final)


In [10]:
obj2_final.show(10)

+----------------------+------+------------+----------+----+--------+----------+--------+--------+-------+--------+
|SubcategoriaEnfermedad|  Zona|Departamento|       Mes|Anio|Cantidad|subcat_idx|zona_idx|dpto_idx|mes_idx|anio_idx|
+----------------------+------+------------+----------+----+--------+----------+--------+--------+-------+--------+
|  ADIPOSIDAD LOCALI...| RURAL|       CUSCO|   OCTUBRE|2020|       2|       0.0|     1.0|     9.0|    1.0|     3.0|
|  SECUELAS DE HIPER...|URBANA|      CALLAO|      MAYO|2017|       6|       7.0|     0.0|     3.0|    5.0|     0.0|
|       DIABETES TIPO 2|URBANA|   CAJAMARCA|     ENERO|2019|       8|       1.0|     0.0|     6.0|   11.0|     2.0|
|          HIGADO GRASO| RURAL|      LORETO|     JUNIO|2016|       5|      10.0|     1.0|    21.0|    8.0|     1.0|
|  SINDROME DE APNEA...| RURAL|     HUÁNUCO|SEPTIEMBRE|2019|       4|       8.0|     1.0|     7.0|    4.0|     2.0|
|  SECUELAS DE HIPER...|URBANA| LA LIBERTAD|   FEBRERO|2020|       5|   

In [11]:
assembler = VectorAssembler(inputCols = ['subcat_idx','zona_idx','dpto_idx','mes_idx','anio_idx'],outputCol='features' )
obj2_final= assembler.transform(obj2_final)

In [12]:
obj2_final = obj2_final.withColumn("label", obj2_final["Cantidad"])
obj2_final.show(10)

+----------------------+------+------------+----------+----+--------+----------+--------+--------+-------+--------+--------------------+-----+
|SubcategoriaEnfermedad|  Zona|Departamento|       Mes|Anio|Cantidad|subcat_idx|zona_idx|dpto_idx|mes_idx|anio_idx|            features|label|
+----------------------+------+------------+----------+----+--------+----------+--------+--------+-------+--------+--------------------+-----+
|  ADIPOSIDAD LOCALI...| RURAL|       CUSCO|   OCTUBRE|2020|       2|       0.0|     1.0|     9.0|    1.0|     3.0|[0.0,1.0,9.0,1.0,...|    2|
|  SECUELAS DE HIPER...|URBANA|      CALLAO|      MAYO|2017|       6|       7.0|     0.0|     3.0|    5.0|     0.0|[7.0,0.0,3.0,5.0,...|    6|
|       DIABETES TIPO 2|URBANA|   CAJAMARCA|     ENERO|2019|       8|       1.0|     0.0|     6.0|   11.0|     2.0|[1.0,0.0,6.0,11.0...|    8|
|          HIGADO GRASO| RURAL|      LORETO|     JUNIO|2016|       5|      10.0|     1.0|    21.0|    8.0|     1.0|[10.0,1.0,21.0,8....|    5|

In [13]:
obj2_train, obj2_test = obj2_final.randomSplit([0.8, 0.2], seed=23)

# Aplicación del modelo del algortimo

In [14]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()
regresion_model = lr.fit(obj2_train)

In [15]:
prediction = regresion_model.transform(obj2_test)
prediction.select("features","label","prediction").show(5,False)

+-----------------------+-----+-----------------+
|features               |label|prediction       |
+-----------------------+-----+-----------------+
|[0.0,1.0,17.0,6.0,1.0] |6    |4.982421879595507|
|(5,[1,2],[1.0,17.0])   |4    |4.959633983275003|
|[0.0,1.0,17.0,0.0,4.0] |7    |4.952056363186445|
|[0.0,1.0,17.0,0.0,5.0] |7    |4.950161958164306|
|[0.0,1.0,17.0,11.0,5.0]|6    |4.995412843959153|
+-----------------------+-----+-----------------+
only showing top 5 rows



In [16]:
print("Coefficients: " + str(regresion_model.coefficients))
print("Intercept: " + str(regresion_model.intercept))

Coefficients: [-0.0027749507010601067,-0.014739271361511527,-0.0019226237261641835,0.0041137168904407115,-0.0018944050221393707]
Intercept: 5.007057857981305


In [18]:
from pyspark.sql.functions import round
#Exportar datos pronosticados

df_prediction=prediction.drop('features')
df_prediction.show(10)
df_prediction.coalesce(1).write.option('header',True).csv('prediccion_2.csv')


+----------------------+-----+------------+-------+----+--------+----------+--------+--------+-------+--------+-----+-----------------+
|SubcategoriaEnfermedad| Zona|Departamento|    Mes|Anio|Cantidad|subcat_idx|zona_idx|dpto_idx|mes_idx|anio_idx|label|       prediction|
+----------------------+-----+------------+-------+----+--------+----------+--------+--------+-------+--------+-----+-----------------+
|  ADIPOSIDAD LOCALI...|RURAL|    AMAZONAS|  ABRIL|2016|       6|       0.0|     1.0|    17.0|    6.0|     1.0|    6|4.982421879595507|
|  ADIPOSIDAD LOCALI...|RURAL|    AMAZONAS| AGOSTO|2017|       4|       0.0|     1.0|    17.0|    0.0|     0.0|    4|4.959633983275003|
|  ADIPOSIDAD LOCALI...|RURAL|    AMAZONAS| AGOSTO|2018|       7|       0.0|     1.0|    17.0|    0.0|     4.0|    7|4.952056363186445|
|  ADIPOSIDAD LOCALI...|RURAL|    AMAZONAS| AGOSTO|2021|       7|       0.0|     1.0|    17.0|    0.0|     5.0|    7|4.950161958164306|
|  ADIPOSIDAD LOCALI...|RURAL|    AMAZONAS|  ENE

# Evaluación del modelo de algoritmo

In [19]:
from pyspark.ml.evaluation import RegressionEvaluator

#Evaluando al modelo con las metricas de R cuadrado , Error cuadratico medio (MSE), Error absoluto medio(MAE), Raiz del error cuadratico medio(RMSE)
#MSE: Promedio de la diferencia cuadratica entre los valores original y pronosticado
#MAE: Promedio absoluto de la diferencia absoluta entre los valores reales y pronosticados
#RMSE: Mide la desviacion estandar de los residuos
#R2: Proporcion de la varianza en la variable dependiente que es explicada por el modelo de regresion lineal (siempre < 1)

evaluator = RegressionEvaluator()
print(evaluator.evaluate(prediction,
{evaluator.metricName: "r2"})
)
print(evaluator.evaluate(prediction,
{evaluator.metricName: "mse"})
)
print(evaluator.evaluate(prediction,
{evaluator.metricName: "rmse"})
)
print(evaluator.evaluate(prediction,
{evaluator.metricName: "mae"})
)

-0.00031422430740501994
4.90581658817532
2.2149078057958347
1.7381515664296157
